In [4]:
file_path = "C:\\Users\\arvin\\OneDrive\\Documents\\GitHub\\Projects\\Python Projects\\Non-GUI Projects\\Time Planner\\Test.csv"

In [12]:
import pandas as pd
from datetime import datetime, timedelta

def normalize(column):
    """ Normalize the column to a range between 0 and 1 """
    return (column.max() - column) / (column.max() - column.min())

def convert_importance(importance):
    """ Convert categorical importance to numerical values """
    mapping = {
        "Very Important": 3,
        "Neutral": 2,
        "Less Important": 1
    }
    return importance.map(mapping)

def calculate_remaining_time(deadline_dates, deadline_times):
    """ Calculate remaining time from now until the deadline """
    current_datetime = datetime.now()
    deadline_datetimes = pd.to_datetime(deadline_dates + ' ' + deadline_times, format='%Y-%m-%d %I:%M %p')
    return (deadline_datetimes - current_datetime).dt.total_seconds() / 3600  # Convert seconds to hours

def calculate_daily_plan(df, days=7):
    """ Calculate the daily plan for each task over the specified number of days """
    # Calculate remaining time for the deadlines
    df['Remaining Time'] = calculate_remaining_time(df['Expected Deadline'], df['Actual Time to Submit'])
    
    # Normalize the columns
    df['ED_norm'] = normalize(df['Remaining Time'])
    df['ETC_norm'] = normalize(df['Expected Time to Complete'])
    df['RT_norm'] = df['Recurring Task'].apply(lambda x: 0 if x.lower() == 'yes' else 1)
    df['DI_norm'] = normalize(convert_importance(df['Degree of Importance']))
    df['Amount of Time to Give'] = df['Amount of Time to Give'].fillna(0)  # Fill NaN values with 0
    
    # Calculate the total weight for each task
    df['Total Weight'] = df['ED_norm'] + df['ETC_norm'] + df['RT_norm'] + df['DI_norm']
    
    # Calculate the daily plan
    daily_plan = {}
    current_date = datetime.now()
    for i in range(days):
        daily_plan[current_date.strftime('%A')] = {}
        total_available_time = 12  # Max hours in a day
        total_recurring_time = df[df['Recurring Task'].str.lower() == 'yes']['Amount of Time to Give'].sum()
        total_non_recurring_weight = df[(df['Recurring Task'].str.lower() == 'no') & (df['Work Name'] != 'Task C')]['Total Weight'].sum()
        for index, row in df.iterrows():
            work_name = row['Work Name']
            weight = row['Total Weight']
            if row['Recurring Task'].lower() == 'yes':
                daily_time = row['Amount of Time to Give']
            else:
                proportion_time = weight / total_non_recurring_weight
                daily_time = total_available_time * proportion_time
            daily_plan[current_date.strftime('%A')][work_name] = min(daily_time, total_available_time)  # Ensure task time does not exceed max available time
        current_date += timedelta(days=1)  # Move to the next day
    return daily_plan

def main():
    # Load the CSV file
    #file_path = input("Enter the path to the CSV file: ")
    df = pd.read_csv(file_path)

    # Calculate the daily plan for the tasks over the next 7 days
    daily_plan = calculate_daily_plan(df)

    # Output the daily plan
    print("Daily Plan for the next 7 days:")
    for day, tasks in daily_plan.items():
        print(f"On {day}:")
        for task, time in tasks.items():
            print(f"- {task}: {time:.2f} hours")

if __name__ == "__main__":
    main()


Daily Plan for the next 7 days:
On Wednesday:
- Task A: 4.89 hours
- Task B: 1.00 hours
- Task C: 8.82 hours
- Task D: 7.11 hours
- Task E: 1.00 hours
On Thursday:
- Task A: 4.89 hours
- Task B: 1.00 hours
- Task C: 8.82 hours
- Task D: 7.11 hours
- Task E: 1.00 hours
On Friday:
- Task A: 4.89 hours
- Task B: 1.00 hours
- Task C: 8.82 hours
- Task D: 7.11 hours
- Task E: 1.00 hours
On Saturday:
- Task A: 4.89 hours
- Task B: 1.00 hours
- Task C: 8.82 hours
- Task D: 7.11 hours
- Task E: 1.00 hours
On Sunday:
- Task A: 4.89 hours
- Task B: 1.00 hours
- Task C: 8.82 hours
- Task D: 7.11 hours
- Task E: 1.00 hours
On Monday:
- Task A: 4.89 hours
- Task B: 1.00 hours
- Task C: 8.82 hours
- Task D: 7.11 hours
- Task E: 1.00 hours
On Tuesday:
- Task A: 4.89 hours
- Task B: 1.00 hours
- Task C: 8.82 hours
- Task D: 7.11 hours
- Task E: 1.00 hours


In [15]:
df = pd.read_csv(file_path)
df

,Work Name,Expected Deadline,Actual Time to Submit,Expected Time to Complete,Recurring Task,Degree of Importance,Amount of Time to Give
0,Task A,2024-06-20,2:00 PM,5,No,Very Important,2
1,Task B,2024-06-12,9:00 AM,2,Yes,Less Important,1
2,Task C,2024-06-25,5:00 PM,1,No,Neutral,1
3,Task D,2024-06-15,12:00 PM,4,No,Very Important,2
4,Task E,2024-06-22,4:00 PM,3,Yes,Neutral,1
